# MAIN

In [1]:
import tensorflow as tf
import numpy as np
import model_tf2.net_factory as netf
import os

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
#init some global variables
num_train_files = 128 #number of training tfrecords
num_val_files = 64 #number of testing tfrecords
buffer_size = 100
num_channels = 3
img_size = 64
num_classes = 200

In [4]:
def get_filenames(is_training):
    """
    input: is_training 
    output: a list of training/validation file names
    """
    filenames = []
    if is_training:
        for i in range(num_train_files):
            filename = "data/tf_records/train/" + 'train-%05d-of-00128' % i
            filenames.append(filename)
    else:
        for i in range(num_val_files):
            filename = "data/tf_records/val/" + 'val-%05d-of-00064' % i
            filenames.append(filename)
    return filenames

def parse_record(record):
    """
    input: a tfrecord
    output: parsed tfrecord based on features
    """
    name_to_features = {
        'image/class/label': tf.io.FixedLenFeature([], tf.int64),
        'image/class/synset': tf.io.FixedLenFeature([], tf.string),
        'image/encoded': tf.io.FixedLenFeature([], tf.string)
    }
    return tf.io.parse_single_example(record, name_to_features)

def preprocess_data(is_training):
    """
    input: bool is_training
    output: training/val X,y tuple ready to feed into models
    """
    #init X and y list
    X = []
    y = []
    #get all filenames
    filenames = get_filenames(is_training)
    raw_dataset = tf.data.TFRecordDataset(filenames)
    parsed_dataset = raw_dataset.map(parse_record)
    num_samples = None
    if is_training:
        num_samples = num_train_files*buffer_size
    else:
        num_samples = num_val_files*buffer_size
        
    print(num_samples)
    #get image and label from each parsed sample
    for parsed in parsed_dataset.take(num_samples):
        image = tf.io.decode_jpeg(parsed['image/encoded'], channels=3)
        label = parsed["image/class/label"]
        X.append(image)
        y.append(label)
    #reshape X to num_samples * num_channel * height * width 
    #cast to float 32
    #X = tf.image.convert_image_dtype(X, dtype=tf.float32, saturate=False)
    X = tf.reshape(tf.stack(X), (num_samples, num_channels, img_size, img_size))
    X = X/255
    #X = tf.cast(tf.reshape(tf.stack(X), 
                   #(num_samples, num_channels, img_size, img_size)),
                #tf.float32)
    y = tf.stack(y)
    #y = tf.cast(tf.stack(y), tf.float32)
    return X, y


In [5]:
def format_image(image):
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.transpose(tf.reshape(image,(img_size, img_size, num_channels)), [2,0,1])
    image /= 255.
    return image

def read_parsed(parsed):
    image = format_image(parsed['image/encoded'])
    label = parsed['image/class/label'] - 1
    
    return image, label
    
def get_dataset(filenames, batch_size=32):
    
    raw_dataset = tf.data.TFRecordDataset(filenames)
    parsed_dataset = raw_dataset.map(parse_record)
    dataset = parsed_dataset.map(read_parsed)
    
    dataset = dataset.shuffle(84)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    
    return dataset.repeat()

In [6]:
def create_model():
    model = netf.myfishnet()
    model.compile(optimizer="adam",
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def train(model, batch_size=256, epochs=10, cpnum=0):
    
    train_files = get_filenames(is_training=True)
    val_files = get_filenames(is_training=False)

    train_ds = get_dataset(train_files, batch_size=batch_size)
    val_ds = get_dataset(val_files, batch_size=batch_size)

    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
    
    #save checkpoints for quicker access later
    checkpoint_path = f"checkpoints/training_{cpnum}/cp.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)
    
    # train model with cp_callback
    model.fit(
        train_ds, validation_data=val_ds,
        epochs=epochs, steps_per_epoch=100, validation_steps=50,
        callbacks = [lr_reducer, cp_callback]
    )

    return model

In [7]:
tf.config.run_functions_eagerly(True)

cpnum = 0

model = create_model()
model = train(model, batch_size=256, epochs=5, cpnum=cpnum)

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/15
100/100 [==============================] - 60s 546ms/step - loss: 5.1784 - accuracy: 0.0225 - val_loss: 5.5376 - val_accuracy: 0.0052

Epoch 00001: saving model to checkpoints/training_0/cp.ckpt
Epoch 2/15
100/100 [==============================] - 68s 685ms/step - loss: 4.6620 - accuracy: 0.0620 - val_loss: 5.9585 - val_accuracy: 0.0053

Epoch 00002: saving model to checkpoints/training_0/cp.ckpt
Epoch 3/15
100/100 [==============================] - 94s 943ms/step - loss: 4.3548 - accuracy: 0.0965 - val_loss: 6.2686 - val_accuracy: 0.0053

Epoch 00003: saving model to checkpoints/training_0/cp.ckpt
Epoch 4/15
100/100 [==============================] - 132s 1s/step - loss: 4.1677 - accuracy: 0.1200 - val_loss: 6.4514 - val_accuracy: 0.0052

Epoch 00004: saving model to checkpoints/training_0/cp.ckpt
Epoch 5/15
100/100 [==============================] - 178s 2s/step - loss: 3.9844 - accuracy: 0.1431 - val_loss: 6.4902 - val_accuracy: 0.0051

Epoch 00005: saving model to check

In [10]:
# myfishnet()
cpnum = 0
while cpnum < 10:
    model = create_model()
    model.load_weights(f'checkpoints/training_{cpnum}/cp.ckpt')
    cpnum += 1
    model = train(model, batch_size=256, epochs=5, cpnum=cpnum)

Epoch 1/5
100/100 [==============================] - 55s 539ms/step - loss: 3.1648 - accuracy: 0.2733 - val_loss: 4.0154 - val_accuracy: 0.1417

Epoch 00001: saving model to checkpoints/training_1/cp.ckpt
Epoch 2/5
100/100 [==============================] - 67s 677ms/step - loss: 3.0287 - accuracy: 0.2985 - val_loss: 4.0230 - val_accuracy: 0.1482

Epoch 00002: saving model to checkpoints/training_1/cp.ckpt
Epoch 3/5
100/100 [==============================] - 92s 928ms/step - loss: 2.9887 - accuracy: 0.3073 - val_loss: 3.7970 - val_accuracy: 0.1767

Epoch 00003: saving model to checkpoints/training_1/cp.ckpt
Epoch 4/5
100/100 [==============================] - 128s 1s/step - loss: 3.0631 - accuracy: 0.2908 - val_loss: 3.6048 - val_accuracy: 0.2002

Epoch 00004: saving model to checkpoints/training_1/cp.ckpt
Epoch 5/5
100/100 [==============================] - 175s 2s/step - loss: 2.9800 - accuracy: 0.3064 - val_loss: 3.6012 - val_accuracy: 0.2085

Epoch 00005: saving model to checkpoint

100/100 [==============================] - 55s 534ms/step - loss: 0.9832 - accuracy: 0.7626 - val_loss: 4.7908 - val_accuracy: 0.1670

Epoch 00001: saving model to checkpoints/training_9/cp.ckpt
Epoch 2/5
100/100 [==============================] - 67s 674ms/step - loss: 1.7586 - accuracy: 0.5530 - val_loss: 4.4434 - val_accuracy: 0.1955

Epoch 00002: saving model to checkpoints/training_9/cp.ckpt
Epoch 3/5
100/100 [==============================] - 92s 925ms/step - loss: 2.0740 - accuracy: 0.4703 - val_loss: 4.2951 - val_accuracy: 0.1971

Epoch 00003: saving model to checkpoints/training_9/cp.ckpt
Epoch 4/5
100/100 [==============================] - 128s 1s/step - loss: 2.1121 - accuracy: 0.4625 - val_loss: 4.2275 - val_accuracy: 0.2032

Epoch 00004: saving model to checkpoints/training_9/cp.ckpt
Epoch 5/5
100/100 [==============================] - 175s 2s/step - loss: 1.1582 - accuracy: 0.7074 - val_loss: 4.6112 - val_accuracy: 0.1721

Epoch 00005: saving model to checkpoints/training

## myfishnet2

In [6]:
def create_model():
    model = netf.myfishnet2()
    model.compile(optimizer="adam",
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def train(model, batch_size=256, epochs=10, cpnum=0):
    
    train_files = get_filenames(is_training=True)
    val_files = get_filenames(is_training=False)

    train_ds = get_dataset(train_files, batch_size=batch_size)
    val_ds = get_dataset(val_files, batch_size=batch_size)

    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
    
    #save checkpoints for quicker access later
    checkpoint_path = f"fishnet2/training_{cpnum}/cp.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)
    
    # train model with cp_callback
    model.fit(
        train_ds, validation_data=val_ds,
        epochs=epochs, steps_per_epoch=100, validation_steps=50,
        callbacks = [lr_reducer, cp_callback]
    )

    return model

In [7]:
tf.config.run_functions_eagerly(True)

cpnum = 0

model = create_model()
model = train(model, batch_size=256, epochs=5, cpnum=cpnum)

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/5
100/100 [==============================] - 95s 886ms/step - loss: 5.1834 - accuracy: 0.0196 - val_loss: 5.4569 - val_accuracy: 0.0054

Epoch 00001: saving model to fishnet2/training_0/cp.ckpt
Epoch 2/5
100/100 [==============================] - 101s 1s/step - loss: 4.7077 - accuracy: 0.0604 - val_loss: 5.7744 - val_accuracy: 0.0048

Epoch 00002: saving model to fishnet2/training_0/cp.ckpt
Epoch 3/5
100/100 [==============================] - 126s 1s/step - loss: 4.3995 - accuracy: 0.0935 - val_loss: 6.1829 - val_accuracy: 0.0048

Epoch 00003: saving model to fishnet2/training_0/cp.ckpt
Epoch 4/5
100/100 [==============================] - 164s 2s/step - loss: 4.2022 - accuracy: 0.1164 - val_loss: 6.5068 - val_accuracy: 0.0053

Epoch 00004: saving model to fishnet2/training_0/cp.ckpt
Epoch 5/5
100/100 [==============================] - 209s 2s/step - loss: 4.0201 - accuracy: 0.1420 - val_loss: 6.6597 - val_accuracy: 0.0073

Epoch 00005: saving model to fishnet2/training_0/cp.ckp

In [8]:
# myfishnet2()
cpnum = 0
while cpnum < 6:
    model = create_model()
    model.load_weights(f'fishnet2/training_{cpnum}/cp.ckpt')
    cpnum += 1
    model = train(model, batch_size=256, epochs=5, cpnum=cpnum)

Epoch 1/5
100/100 [==============================] - 90s 877ms/step - loss: 3.7775 - accuracy: 0.1747 - val_loss: 6.7939 - val_accuracy: 0.0054

Epoch 00001: saving model to fishnet2/training_1/cp.ckpt
Epoch 2/5
100/100 [==============================] - 102s 1s/step - loss: 3.7979 - accuracy: 0.1683 - val_loss: 6.6053 - val_accuracy: 0.0079

Epoch 00002: saving model to fishnet2/training_1/cp.ckpt
Epoch 3/5
100/100 [==============================] - 127s 1s/step - loss: 3.6678 - accuracy: 0.1912 - val_loss: 6.5040 - val_accuracy: 0.0108

Epoch 00003: saving model to fishnet2/training_1/cp.ckpt
Epoch 4/5
100/100 [==============================] - 165s 2s/step - loss: 3.5830 - accuracy: 0.2020 - val_loss: 6.7457 - val_accuracy: 0.0088

Epoch 00004: saving model to fishnet2/training_1/cp.ckpt
Epoch 5/5
100/100 [==============================] - 213s 2s/step - loss: 3.4709 - accuracy: 0.2188 - val_loss: 5.8774 - val_accuracy: 0.0245

Epoch 00005: saving model to fishnet2/training_1/cp.ckp